# Final Model Construction

Below is the construction for a multiple linear regression model predicting housing prices in King County. Data is imported from the preprocessed data that was cleaned in preceding notebooks and saved as separate csv files. The data is imported to this notebook and combined for the first model. Further iterations are contained in additional subsections of this notebook.

### Import Packages

In [1]:
import scipy.stats as stats
import pandas as pd
import statsmodels as sm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Import and Combine Cleaned Data

In [ ]:
categorical = pd.read_csv("./data/")
numeric = pd.read_csv("./data/")

### Construct Model

### Evaluate Model Performance

### Assess Potential Model Improvements

# 2nd Model